In [ ]:
from google.colab import drive
ROOT = "/content/drive"  
drive.mount(ROOT)  
%cd "/content/drive/MyDrive/PFE_PIMM/"
%ls

Mounted at /content/drive
/content/drive/MyDrive/PFE_PIMM
 batch_labeling.ipynb       labeling.ipynb          tomo_labels/
'Copie de labeling.ipynb'   line_to_continuum.mat   visualisation.ipynb
 ct_scan/                   meltpool_bitmap/
 extracting.ipynb           registration.ipynb


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# IMPORT THE DATA : 

In [ ]:
import os
import numpy as np
import pandas as pd

TOMO_PATH = os.path.join("tomo_labels", "dataset1")

def load_tomo_data(file_number, tomo_path=TOMO_PATH):
    csv_path = os.path.join(tomo_path, "label_Ec"+file_number+"."+"dat")
    return pd.read_csv(csv_path, delimiter=" ")


pieces = ['0','1']#, '2', '3', '4', '5', '6', '7', '8']
piece = {name: load_tomo_data(name) for name in pieces}

num_slice_reel = {'0': 4302, '1': 4266}


# I. DATA CLEANING
Cleaning process is done by

*   Extacting relevent information : 3D locations (Xg-Yg-Zg) 

*   and size(Volmarch and spherosity) of each porosity 
Eliminating noises 

In [ ]:
def map_max(x, max_slice, max_layer):
  return int((x) * (max_layer) / (max_slice)) + 1

def map_minmax(x, in_min, in_max, out_min, out_max):
  return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)

def get_layer(x, num_slice_reel) :
  x = map_minmax(x,0,num_slice_reel+1,1,417)
  return x

In [ ]:
def clean_tomo_data(df, slices):
  df = df[['xg', 'yg', 'zg', 'volpix','volmarch', 'sphericity']]
  df = df.sort_values('zg', ascending=False)
  df['numLayer'] = df['zg'].apply(get_layer, args=(), num_slice_reel=slices)
  df.numLayer = df.numLayer.astype(int)
  df = df[ df["volpix"] != df["volpix"].max()]
  return df

In [ ]:
newPiece = {num: clean_tomo_data(df, num_slice_reel[num]) for num, df in piece.items()}

# II. DATA LABELING
## 1. Computing indicators and porosity level


In [ ]:
from google.colab import data_table
data_table.DataTable(newPiece['0'])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from scipy.spatial.distance import pdist

vol_layer = 8691600.00

def dist(df):
  return sum(pdist(df.values, 'euclid'))
  
def compute_indicators_porosity_level(df):
  # occurrence frequency
  mu1 = pd.DataFrame(df.pivot_table(index=['numLayer'], aggfunc='size')).rename(columns = {0:'mu1'})
  mu1_norm = (mu1-mu1.mean())/mu1.std()

  # propotion of affected zone 
  mu2 = pd.DataFrame(df.groupby('numLayer')['volmarch'].sum().div(vol_layer)).rename(columns = {'volmarch':'mu2'})
  mu2_norm = (mu2-mu2.mean())/mu2.std()

  # average distance between pores
  mu1_serie = df.pivot_table(index=['numLayer'], aggfunc='size')
  mu3 = pd.DataFrame(df.groupby('numLayer')[['xg','yg']].apply(lambda x: dist(x)).div(mu1_serie)).rename(columns = {0:'mu3'})
  mu3_norm = (mu3-mu3.mean())/mu3.std()

  # porosity level
  mu = pd.DataFrame(mu2_norm['mu2'].mul(mu1_norm['mu1']).div(mu3_norm['mu3'])).rename(columns = {0:'mu'})

  # mu = mu['mu'].apply(lambda x: x*900*900)
  # mu[mu == np.inf] = 1

  # normalised porosity level
  normalised_mu = (mu-mu.min())/(mu.max()-mu.min())
  normalised_mu_V2 = (mu-mu.mean())/mu.std()
  new_df = pd.concat([mu1, mu2, mu3,normalised_mu], axis=1)
  return new_df


In [ ]:
new_df = compute_indicators_porosity_level(newPiece['0'])
data_table.DataTable(new_df)

,mu1,mu2,mu3,mu
numLayer,,,,
1,212,0.000706,50207.640243,0.237990
2,127,0.000075,30621.247225,0.128399
3,144,0.002678,32317.330059,0.620762
4,143,0.005002,33618.125651,1.000000
5,157,0.000108,36619.585557,0.134158
...,...,...,...,...
412,97,0.000020,23477.808461,0.118864
413,94,0.000004,23998.294242,0.117235
414,125,0.000005,29843.827636,0.116234
